In [63]:
# Read data
import pickle
def load_file(filename):
    with open(filename, 'rb') as fi:
        data = pickle.load(fi)
    return data

In [ ]:
from Data_Preprocessing import general as dp
from Data_Preprocessing import calculate_y as cy
from Data_Preprocessing import Train_test

filename = 'C:/Users/USER/Desktop/AIML/資料/Data_cluster'
data = load_file(filename)

cluster = data.groupby(['cluster'])
cluster.get_group(1)

In [ ]:
# data = data.drop(['START_DATE'], axis = 1)
data_train, data_test = dp.split_traintest(data, 0.2)
data_X, data_y = dp.separate_Xy(data_train)
data_Xtest, data_ytest = dp.separate_Xy(data_test)
#outlier_pred, data_X, data_y = dp.outlier_Isolation(data_X, data_y)#another outlier method
outlier_pred, data_X, data_y = dp.outlier_std(data_X, data_y, 4)
data_y = cy.calculate_ymean(data_y)
data_ytest = cy.calculate_ymean(data_ytest)

# Cluster by k mean

In [113]:
cluster = data.groupby(['cluster'])
def clusterdata(c):
    datac = cluster.get_group(c)
    # data = data.drop(['START_DATE'], axis = 1)
    data_train, data_test = dp.split_traintest(datac, 0.2)
    data_X, data_y = dp.separate_Xy(data_train)
    data_Xtest, data_ytest = dp.separate_Xy(data_test)
    #outlier_pred, data_X, data_y = dp.outlier_Isolation(data_X, data_y)#another outlier method
    outlier_pred, data_X, data_y = dp.outlier_std(data_X, data_y, 4)
    data_y = cy.calculate_ymean(data_y)
    data_ytest = cy.calculate_ymean(data_ytest)
    return data_X, data_y, data_Xtest, data_ytest


In [122]:
from sklearn.ensemble import RandomForestRegressor
from Data_Preprocessing import Train_test
import pandas as pd
import numpy as np
# criterion: {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}
for i in range(10):
    data_X, data_y, data_Xtest, data_ytest = clusterdata(i)
    rnd_clf = RandomForestRegressor(n_estimators=10, max_leaf_nodes=20,# oob_score = True
        n_jobs=-1, random_state=42, criterion='poisson')
    rnd_clf.fit(data_X, data_y['y_mean'])
    y_pred = rnd_clf.predict(data_Xtest)
    loss = Train_test.loss_f(y_pred, data_ytest['y_mean'])
    print(loss)

85448.79728088823
81063.25695259195
92074.78936960561
80912.34629371493
91637.6385684662
83457.41157208766
77573.53949457582
102156.34612247418
80206.78536938397
119671.09384219252


# Bad Model

In [66]:
from sklearn.ensemble import RandomForestRegressor
from Data_Preprocessing import Train_test
import pandas as pd
import numpy as np

# criterion: {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}
rnd_clf = RandomForestRegressor(n_estimators=10, max_leaf_nodes=20,# oob_score = True
    n_jobs=-1, random_state=42, criterion='poisson')
rnd_clf.fit(data_X, data_y['y_mean'])
y_pred = rnd_clf.predict(data_Xtest)
loss = Train_test.loss_f(y_pred, data_ytest['y_mean'])


In [67]:
y_predX = rnd_clf.predict(data_X)
distance = np.abs(y_predX - data_y['y_mean'])
sample_performance = []
for i in distance:
    if i >= 10000:
        sample_performance.append(1)
    else:
        sample_performance.append(0)
data_Xperformance = data_X
data_Xperformance['sample_performance'] = sample_performance



In [68]:
cluster = data_Xperformance.groupby(['cluster'])
cluster_preformance = pd.DataFrame()
cluster_preformance['cluster'] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
badprediction_list = []
for i in range(10):
    
    n_badprediction = 0
    for k in cluster.get_group(i)['sample_performance']:
        if k == 1:
            n_badprediction+=1
    badprediction_list.append(n_badprediction)
    # print(f"{i}:{n_badprediction}")
cluster_preformance['n'] = badprediction_list
cluster_preformance

            

,cluster,n
0,0,369
1,1,633
2,2,1234
3,3,579
4,4,240
5,5,781
6,6,1167
7,7,613
8,8,444
9,9,139


In [69]:
cluster_preformance['n'].nlargest(5)

2    1234
6    1167
5     781
1     633
7     613
Name: n, dtype: int64

In [ ]:
# data_X = data_X.drop(['sample_performance'], axis=1)
data_X1 = data_X[data_X['cluster'] != 6]
data_X1 = data_X1[data_X['cluster'] != 2]
data_X1 = data_X1[data_X['cluster'] != 5]
data_X1 = data_X1[data_X['cluster'] != 1]
data_X1 = data_X1[data_X['cluster'] != 7]
data_X1 = data_X1.drop(['sample_performance'], axis=1)
# data_X1
data_y1 = data_y['y_mean'][data_X['cluster'] != 6]
data_y1 = data_y1[data_X['cluster'] != 2]
data_y1 = data_y1[data_X['cluster'] != 5]
data_y1 = data_y1[data_X['cluster'] != 1]
data_y1 = data_y1[data_X['cluster'] != 7]

In [87]:
len(data_X1)

2677

In [ ]:
data_X1test = data_Xtest[data_Xtest['cluster'] != 6]
data_X1test = data_X1test[data_Xtest['cluster'] != 2]
data_X1test = data_X1test[data_Xtest['cluster'] != 5]
data_X1test = data_X1test[data_Xtest['cluster'] != 1]
data_X1test = data_X1test[data_Xtest['cluster'] != 7]
# data_X1test
data_y1test = data_ytest['y_mean'][data_Xtest['cluster'] != 6]
data_y1test = data_y1test[data_Xtest['cluster'] != 2]
data_y1test = data_y1test[data_Xtest['cluster'] != 5]
data_y1test = data_y1test[data_Xtest['cluster'] != 1]
data_y1test = data_y1test[data_Xtest['cluster'] != 7]

In [80]:
data_X1

,START_DATE,AGENT_AGE,ON_BOARD_AGE,SENIORITY,AGENT_SEX,UNIT_POST_CODE,LOCATION_ID,login_mean,contact_mean,contact_custmean,...,CITY_新竹縣,CITY_桃園市,CITY_澎湖縣,CITY_花蓮縣,CITY_苗栗縣,CITY_金門縣,CITY_雲林縣,CITY_高雄市,cluster,sample_performance
18076,865.0,24.0,23.4,0.6,0.0,802334.0,3.0,0.083333,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1
1980,810.0,30.2,25.0,5.2,0.0,800204.0,3.0,5.000000,38.000000,30.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1
3476,855.0,37.7,36.3,1.5,0.0,802406.0,3.0,4.608698,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0
4838,691.0,60.8,45.7,15.1,0.0,802406.0,3.0,8.416667,32.666667,28.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0
17409,857.0,22.9,21.6,1.3,1.0,801731.0,3.0,0.166667,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18029,865.0,28.5,27.9,0.6,0.0,831141.0,3.0,0.416667,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,0
12155,735.0,46.0,34.5,11.5,1.0,104471.0,1.0,0.666667,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0
12133,760.0,38.7,29.3,9.4,1.0,110407.0,1.0,4.608698,22.970611,19.323540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1
19614,869.0,23.1,22.8,0.3,1.0,814018.0,3.0,0.583333,1.083333,1.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [89]:
rnd_clf = RandomForestRegressor(n_estimators=10, max_leaf_nodes=20,# oob_score = True
    n_jobs=-1, random_state=42, criterion='poisson')
rnd_clf.fit(data_X1, data_y1)
y_pred = rnd_clf.predict(data_X1test)
loss = Train_test.loss_f(y_pred, data_y1test)
loss


85316.30951481959